In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)
from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


c:/Users/Admin/Data/ads_fraud_detection


# Lựa chọn đặc trưng

### (1) Tạo thư mục lưu trữ (nếu chưa có)

In [2]:
save_dir=f"{exps_dir}/exp1/exp_adasyn"

os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
x_train=pd.read_excel(f'{save_dir}/x_train.xlsx')
y_train=pd.read_excel(f'{save_dir}/y_train.xlsx')
x_test=pd.read_excel(f'{save_dir}/x_test.xlsx')
y_test=pd.read_excel(f'{save_dir}/y_test.xlsx')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(998, 39) (998, 1) (330, 39) (330, 1)


### (2) Lựa chọn các đặc trưng

In [4]:
x_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
0,1,0,0,1,1,-1.770734,0,1,2,11,6,4,1.270697,0.970481,0,2,2,0,4,1,0,2,0,1,1,1,0.249230,-0.612776,0.358110,0.416329,8,24,2,0.579574,-0.179439,-1.020460,-1.393943,-0.947412,0
1,0,2,1,0,2,-1.979562,0,1,4,11,0,1,-0.919382,0.175400,0,1,0,0,4,4,1,2,0,0,3,0,-0.845210,-0.976821,-0.374783,-0.834115,6,6,0,-0.547936,-0.179439,0.908584,0.115223,0.953085,0
2,1,0,2,0,2,0.675544,0,0,0,5,14,4,-0.919382,-0.691310,2,3,0,0,6,5,2,0,2,2,1,1,1.224890,1.953951,1.988592,0.695152,11,15,1,1.481582,0.108179,-0.744883,0.115223,-0.947412,0
3,1,0,0,0,2,-0.492901,0,0,2,2,5,5,-0.919382,-1.049454,0,3,2,2,1,4,2,1,0,0,2,2,0.260994,0.690017,-0.415957,0.293945,5,14,0,1.368831,-0.467057,1.735317,-1.626123,2.853582,0
4,1,0,2,1,1,0.613672,0,1,4,2,17,0,0.957325,0.970481,2,3,0,3,4,1,1,0,2,1,2,2,0.682604,-0.228278,1.132177,0.723886,10,1,1,0.128570,0.971032,1.735317,1.160031,-0.947412,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,0,0,0,0,1,-0.414506,8,0,4,8,11,1,-0.120137,-0.371848,0,2,1,2,2,3,0,1,1,0,1,0,0.119890,0.119163,0.268703,0.067653,0,22,0,-0.735087,-1.042293,-0.097179,0.333181,-0.300978,0
994,1,0,0,1,2,-1.462109,0,0,4,1,5,2,-0.404621,0.273740,2,0,1,2,2,4,1,0,1,0,2,1,-0.988728,-0.942013,-0.846936,-0.922372,7,8,0,0.143495,-0.152373,-0.469305,-1.311828,-0.212370,0
995,0,0,1,0,1,-0.749460,0,0,3,1,6,0,0.274150,-0.644990,2,1,1,1,5,4,1,0,1,0,2,1,0.312484,-0.154041,-0.133303,0.512686,5,10,0,-0.384960,1.228954,-0.341268,-0.753565,0.953085,0
996,1,0,2,2,1,-0.064613,0,0,0,6,4,1,-0.919382,0.425323,1,2,0,3,4,1,0,0,1,0,1,1,0.380807,-0.235482,1.487158,0.210839,8,5,1,0.400513,-0.288926,-1.068610,0.105988,-0.947412,0


In [5]:
def modelcheck_info(model,name):
    print(name)
    try:
        coefficients = model.coef_.flatten()
    except:
        coefficients = model.feature_importances_.flatten()
    feature_importance = pd.DataFrame({
        'Feature': x_train.columns,
        'Coefficient': coefficients
    })

    print('bi loai bo',feature_importance[feature_importance['Coefficient'] == 0])
    feature_importance = feature_importance[feature_importance['Coefficient'] != 0]
    feature_importance['Absolute Coefficient'] = feature_importance['Coefficient'].abs()
    feature_importance.sort_values(by='Absolute Coefficient', ascending=False, inplace=True)

    print("Xếp hạng các đặc trưng theo mức độ quan trọng:")
    print(feature_importance[['Feature', 'Coefficient']])

In [6]:
models=[]
models.append(LogisticRegression())
models.append(LinearDiscriminantAnalysis())
models.append(DecisionTreeClassifier())
names=['LR','LDA','DTC']
feature_SelectFromModel={}
for model,name in zip(models,names):
    check = model.fit(x_train, y_train)
    selector = SelectFromModel(check, prefit=True)

    selected_features = x_train.columns[(selector.get_support())]
    modelcheck_info(model,name)
    print("Các đặc trưng được chọn:")
    feature_SelectFromModel[name]=list(selected_features)
    print(selected_features)

LR
bi loai bo                Feature  Coefficient
38  year_incident_date          0.0
Xếp hạng các đặc trưng theo mức độ quan trọng:
                        Feature  Coefficient
16            incident_severity    -1.835034
21  number_of_vehicles_involved    -1.016121
0            months_as_customer    -0.923615
14                incident_type    -0.758759
32                    auto_year    -0.405018
27                 injury_claim    -0.359993
3                    policy_csl    -0.333860
1                           age    -0.285895
22              property_damage    -0.258766
36            day_incident_date    -0.248590
28               property_claim     0.234526
4             policy_deductable    -0.225518
15               collision_type    -0.216864
23              bodily_injuries    -0.182505
6                umbrella_limit     0.171221
25      police_report_available    -0.167357
17        authorities_contacted    -0.146081
19                incident_city    -0.143424
24          

### Giải thích:

Phương pháp **SelectFromModel** từ thư viện sklearn.feature_selection. Phương pháp này hoạt động bằng cách sử dụng một mô hình học máy để đánh giá độ quan trọng của từng đặc trưng và sau đó chọn những đặc trưng quan trọng nhất.

   Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
  
       #### 1. LogisticRegression:
       Các đặc trưng được chọn:
* Index(['age', 'policy_csl', 'insured_education_level', 'insured_occupation',
       'incident_severity', 'incident_state', 'incident_city',
       'incident_hour_of_the_day', 'property_damage', 'witnesses',
       'property_claim', 'vehicle_claim', 'auto_make'],
      dtype='object')
      
   #### 2. LinearDiscriminantAnalysis:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip', 'insured_sex',
       'incident_severity', 'incident_hour_of_the_day', 'property_damage',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')
      
   #### 3. DecisionTreeClassifier:
   Các đặc trưng được chọn:
* Index(['policy_number', 'policy_annual_premium', 'insured_zip',
       'insured_occupation', 'insured_hobbies', 'incident_severity',
       'incident_location', 'police_report_available', 'total_claim_amount',
       'injury_claim', 'month_policy_bind_date', 'day_incident_date'],
      dtype='object')

In [7]:
def feature_selection_inRFE(model,name):
    rfe=RFE(estimator=model, n_features_to_select=10)
    fit=rfe.fit(x_train, y_train)
#     print("Num Features: %d"% fit.n_features_)
#     print("Selected Features: %s" % fit.support_)
#     print("Features Ranking: %s" %fit.ranking_)
#     for index, value in enumerate(fit.ranking_):
#         print(f"{value}: {x_train.columns[index]}")
    sorted_columns = [x for _, x in sorted(zip(fit.ranking_, x_train.columns))]
    sort=sorted(fit.ranking_)

#     for i,j in zip(sort,sorted_columns):
#         print("Columns sorted by RFE ranking",i,":",j)

    return [j for i,j in zip(sort,sorted_columns) if i==1]

In [8]:
models = [
    LogisticRegression(),
    LinearDiscriminantAnalysis(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LGBMClassifier(),  
    XGBClassifier(use_label_encoder=False)  
]
feature_RFE={}

names = ['LR', 'LDA', 'DTC', 'RD', 'LGBM', 'XGB']
for model,name in zip(models,names):
    print("RFE")
    print(f"Kiểm tra mô hình {name}")
    model.fit(x_train,y_train)
    print(feature_selection_inRFE(model,name))
    feature_RFE[name]=feature_selection_inRFE(model,name)

RFE
Kiểm tra mô hình LR
['age', 'auto_year', 'incident_severity', 'incident_type', 'injury_claim', 'months_as_customer', 'number_of_vehicles_involved', 'policy_csl', 'property_damage', 'vehicle_claim']
RFE
Kiểm tra mô hình LDA
['age', 'auto_year', 'incident_severity', 'incident_type', 'injury_claim', 'months_as_customer', 'number_of_vehicles_involved', 'policy_csl', 'property_damage', 'vehicle_claim']
RFE
Kiểm tra mô hình DTC
['auto_make', 'incident_severity', 'injury_claim', 'insured_education_level', 'insured_hobbies', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'vehicle_claim', 'year_policy_bind_date']
RFE
Kiểm tra mô hình RD
['capital-loss', 'day_incident_date', 'incident_severity', 'injury_claim', 'insured_hobbies', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'total_claim_amount', 'vehicle_claim']
RFE
Kiểm tra mô hình LGBM
[LightGBM] [Info] Number of positive: 487, number of negative: 511
[LightGBM] [Info] Auto-choosing row-wise mu

### Giải thích:
**Recursive Feature Elimination (RFE)** là một phương pháp trong lựa chọn đặc trưng (feature selection) được sử dụng trong học máy để giảm số lượng đặc trưng của mô hình mà vẫn giữ được hiệu suất tốt.

Các model được dùng trong phương pháp này:
   * **'LR'**: LogisticRegression
   * **'LDA'**: LinearDiscriminantAnalysis
   * **'DTC'**: DecisionTreeClassifier
   * **'RD'**: RandomForestClassifier

In [9]:
special_models=[KNeighborsClassifier(),GaussianNB(),MLPClassifier(),GradientBoostingClassifier(),MultinomialNB(),SVC()]
names=['KNC','GNB','NN','BG','NB','SVM']
def feature_selection_inKBest(model, name):
    k_best = SelectKBest(score_func=f_classif, k=11)
    fit = k_best.fit(x_train, y_train)
    selected_features = fit.get_support()
    selected_column_names = x_train.columns[selected_features]
    return selected_column_names
feature_Kbest={}
for model,name in zip(special_models,names):
    print(f'Lua chon dac trung {name}')
    print(feature_selection_inKBest(model, name))
    feature_Kbest[name]=list(feature_selection_inKBest(model, name))

Lua chon dac trung KNC
Index(['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city', 'property_damage', 'police_report_available', 'total_claim_amount', 'property_claim', 'vehicle_claim', 'auto_year'], dtype='object')
Lua chon dac trung GNB
Index(['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city', 'property_damage', 'police_report_available', 'total_claim_amount', 'property_claim', 'vehicle_claim', 'auto_year'], dtype='object')
Lua chon dac trung NN
Index(['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city', 'property_damage', 'police_report_available', 'total_claim_amount', 'property_claim', 'vehicle_claim', 'auto_year'], dtype='object')
Lua chon dac trung BG
Index(['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city', 'property_damage', 'police_report_available', 'total_claim_amount', 'property_claim', '

### Giải thích:
Phương pháp **SelectKBest** từ thư viện sklearn.feature_selection để lựa chọn đặc trưng dựa trên kỹ thuật kiểm định thống kê.
 
Các model được dùng trong phương pháp này:
   * **'KNC'**: KNeighborsClassifier
   * **'GNB'**: GaussianNB
   * **'NN'**: MLPClassifier
   * **'BG'**: GradientBoostingClassifier
   * **'NB'**: MultinomialNB
   * **'SVM'**: SVC

In [10]:
print(feature_Kbest)
print(feature_SelectFromModel)
print(feature_RFE)

{'KNC': ['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city', 'property_damage', 'police_report_available', 'total_claim_amount', 'property_claim', 'vehicle_claim', 'auto_year'], 'GNB': ['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city', 'property_damage', 'police_report_available', 'total_claim_amount', 'property_claim', 'vehicle_claim', 'auto_year'], 'NN': ['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city', 'property_damage', 'police_report_available', 'total_claim_amount', 'property_claim', 'vehicle_claim', 'auto_year'], 'BG': ['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city', 'property_damage', 'police_report_available', 'total_claim_amount', 'property_claim', 'vehicle_claim', 'auto_year'], 'NB': ['months_as_customer', 'policy_csl', 'incident_severity', 'authorities_contacted', 'incident_city',

In [11]:
feature_model_selection={**feature_RFE,**feature_Kbest}
np.savez(f'{save_dir}/feature_model_selection.npz', feature_model_selection = feature_model_selection)
feature_model_selection

{'LR': ['age',
  'auto_year',
  'incident_severity',
  'incident_type',
  'injury_claim',
  'months_as_customer',
  'number_of_vehicles_involved',
  'policy_csl',
  'property_damage',
  'vehicle_claim'],
 'LDA': ['age',
  'auto_year',
  'incident_severity',
  'incident_type',
  'injury_claim',
  'months_as_customer',
  'number_of_vehicles_involved',
  'policy_csl',
  'property_damage',
  'vehicle_claim'],
 'DTC': ['incident_city',
  'incident_severity',
  'injury_claim',
  'insured_education_level',
  'insured_hobbies',
  'month_policy_bind_date',
  'policy_annual_premium',
  'property_claim',
  'vehicle_claim',
  'year_policy_bind_date'],
 'RD': ['capital-loss',
  'day_incident_date',
  'incident_severity',
  'injury_claim',
  'insured_hobbies',
  'month_policy_bind_date',
  'policy_annual_premium',
  'property_claim',
  'total_claim_amount',
  'vehicle_claim'],
 'LGBM': ['auto_model',
  'day_incident_date',
  'day_policy_bind_date',
  'injury_claim',
  'insured_hobbies',
  'month_pol

* gom nhóm thông tin về các đặc trưng được chọn từ hai phương pháp lựa chọn đặc trưng khác nhau (RFE và SelectKBest) vào từ điển **feature_model_selection**.

In [12]:
dict(np.load(f'{save_dir}/feature_model_selection.npz',allow_pickle=True))['feature_model_selection']

array({'LR': ['age', 'auto_year', 'incident_severity', 'incident_type', 'injury_claim', 'months_as_customer', 'number_of_vehicles_involved', 'policy_csl', 'property_damage', 'vehicle_claim'], 'LDA': ['age', 'auto_year', 'incident_severity', 'incident_type', 'injury_claim', 'months_as_customer', 'number_of_vehicles_involved', 'policy_csl', 'property_damage', 'vehicle_claim'], 'DTC': ['incident_city', 'incident_severity', 'injury_claim', 'insured_education_level', 'insured_hobbies', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'vehicle_claim', 'year_policy_bind_date'], 'RD': ['capital-loss', 'day_incident_date', 'incident_severity', 'injury_claim', 'insured_hobbies', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'total_claim_amount', 'vehicle_claim'], 'LGBM': ['auto_model', 'day_incident_date', 'day_policy_bind_date', 'injury_claim', 'insured_hobbies', 'month_policy_bind_date', 'policy_annual_premium', 'property_claim', 'vehicle_claim', 'yea